In [118]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns

In [119]:
from tsfresh import extract_features
from tsfresh import select_features
from pandas import Series,DataFrame
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from sklearn.ensemble import AdaBoostRegressor
from tsfresh.utilities.dataframe_functions import impute

In [120]:
import warnings
warnings.filterwarnings('ignore')

## 载入数据

In [121]:
## ##all_leaf_class_name_dict = {cate_id: cate_name}
 
df = pd.read_csv('./50.csv', header=0, encoding='gbk')

df.columns = ['time', 'ds', 'y']

In [122]:
class_df_all = df[['ds', 'y']]
class_df_all
Y=df[['y']]
Y.index = Y.index + 1
Y=Y[0:-1]   ##本来有10000条数据，经过一系列操作之后不知道为什么少了一条，这两行的操作主要是为了让Y的数据格式和y一致，
           ##从而使其后续能够成功地作滑动处理

Y = pd.Series(Y['y'].values, index=Y.index)
Y




1       14.100000
2       14.100000
3       14.100000
4       14.100000
5       14.877778
6       14.100000
7       14.100000
8       14.100000
9       14.100000
10      14.100000
11      14.100000
12      14.100000
13      14.303390
14      15.100000
15      14.462069
16      14.100000
17      14.100000
18      14.100000
19      14.100000
20      14.100000
21      14.500000
22      15.100000
23      15.100000
24      14.333333
25      14.100000
26      15.016667
27      16.983333
28      18.133898
29      19.100000
30      19.800000
          ...    
9970     5.000000
9971     5.400000
9972     6.000000
9973     6.000000
9974     6.000000
9975     6.000000
9976     6.000000
9977     6.000000
9978     6.000000
9979     6.000000
9980     6.000000
9981     6.966667
9982     7.000000
9983     7.000000
9984     7.000000
9985     7.000000
9986     7.000000
9987     7.000000
9988     7.898305
9989     8.000000
9990     8.000000
9991     8.000000
9992     8.000000
9993     8.000000
9994     8

## 滑动处理

In [123]:
df_shift,y = make_forecasting_frame(class_df_all['ds'],kind="gmv", max_timeshift=10, rolling_direction=1)
## 滑动窗口的大小为10，对9999条数据进行id标记，第1条标记为1，第1~2条标记为2，...，第1~10条标记为10，第2~11条标记为11，
##  ...第9990~9999标记为999999
y

1       29.666667
2       30.000000
3       30.577778
4       31.888889
5       32.466667
6       33.777778
7       34.066667
8       35.000000
9       35.133333
10      36.177778
11      37.000000
12      37.508475
13      38.016667
14      39.000000
15      39.000000
16      39.883333
17      40.216667
18      41.275862
19      40.033333
20      32.116667
21      24.633333
22      19.966667
23      17.683333
24      19.603448
25      21.300000
26      23.366667
27      24.983051
28      26.150000
29      26.966667
30      28.433333
          ...    
9970   -31.900000
9971   -31.900000
9972   -31.833333
9973   -31.900000
9974   -31.866667
9975   -31.900000
9976   -31.883333
9977   -31.883333
9978   -31.866667
9979   -31.900000
9980   -31.850000
9981   -31.866667
9982   -31.800000
9983   -31.866102
9984   -31.816667
9985   -31.750000
9986   -31.849153
9987   -28.357627
9988   -19.366667
9989   -20.442373
9990   -26.103390
9991   -26.866667
9992   -19.350000
9993   -13.500000
9994   -19

In [124]:
df_shift

,time,value,id,kind
89946,0.0,28.434783,1,gmv
79948,0.0,28.434783,2,gmv
89947,1.0,29.666667,2,gmv
69951,0.0,28.434783,3,gmv
79949,1.0,29.666667,3,gmv
89948,2.0,30.000000,3,gmv
59955,0.0,28.434783,4,gmv
69952,1.0,29.666667,4,gmv
79950,2.0,30.000000,4,gmv
89949,3.0,30.577778,4,gmv


## 提取时序特征

In [125]:
extracted_features= extract_features(df_shift, column_id="id", column_sort="time", column_value="value", impute_function=impute, 
                     show_warnings=False)


Feature Extraction: 20it [05:37,  8.69s/it]                        
 'value__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"slope"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"slope"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"slope"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"rvalue"'
 'value__agg_linear_tre

## 对提取的特征量过滤

In [126]:
impute(extracted_features)
features_filtered = select_features(extracted_features, Y) 
features_filtered.shape

(9999, 289)

In [127]:
np.savetxt('data50.csv',features_filtered,delimiter=',')

## 特征量筛选

In [133]:
##features_filtered = pd.read_csv('./data26.csv')

In [134]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
X_=features_filtered.values
X_.shape

(9999L, 269L)

In [137]:
    df=DataFrame(X_,columns=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268])
    df

,0,1,2,3,4,5,6,7,8,9,...,259,260,261,262,263,264,265,266,267,268
0,645.0,60.7,0.00000,12.10,46.4,63.0,54.5,0.0000,0.0000,0.0000,...,0.0,0.0000,20.1,4.26,-0.4140,-1.080,0.0000,0.0,0.000000e+00,0.000
1,1290.0,60.7,0.00000,12.10,46.4,63.0,54.5,0.0000,0.0000,0.0000,...,0.0,0.0000,20.1,4.26,-0.4140,-1.080,0.0000,0.0,0.000000e+00,0.000
2,1940.0,60.7,0.00000,12.10,46.4,63.0,54.5,0.0000,0.0000,0.0000,...,0.0,0.0000,20.1,4.26,-0.4140,-1.080,0.0000,0.0,0.000000e+00,1.000
3,2580.0,60.7,0.00000,12.10,46.4,63.0,54.5,0.0000,0.0000,0.0000,...,0.0,0.0000,20.1,4.26,-0.4140,-1.080,0.0000,1.0,0.000000e+00,0.000
4,3230.0,60.7,0.00000,12.10,46.4,63.0,54.5,0.0000,0.0000,0.0000,...,0.0,0.0000,20.1,4.26,-0.4140,-1.080,0.0000,0.0,1.780000e-15,0.000
5,3870.0,60.7,0.00000,12.10,46.4,63.0,54.5,0.0000,0.0000,0.0000,...,6.0,0.0000,20.1,4.26,-0.4140,-1.080,0.0000,1.0,0.000000e+00,1.000
6,4480.0,26.3,0.06590,19.10,19.3,31.8,54.5,0.0000,0.0000,0.0000,...,1.0,-0.1150,20.1,4.26,-0.2410,-1.080,0.0000,1.0,1.530000e-01,0.143
7,5080.0,39.0,0.06120,27.10,28.8,46.4,32.2,0.0000,0.0000,0.0000,...,2.0,-0.1430,20.1,4.26,-0.6740,-1.080,0.0000,1.0,6.890000e-01,1.000
8,5670.0,44.5,0.05580,22.70,33.3,50.6,38.0,0.0000,0.0000,0.1150,...,3.0,-0.1250,20.1,4.26,0.4390,-1.130,-0.1150,0.0,1.110000e+00,1.000
9,6270.0,55.6,0.05110,11.50,42.5,57.7,49.8,0.0000,0.0000,0.1110,...,4.0,-0.1110,20.1,4.26,-0.4000,-1.100,-0.1110,2.0,1.100000e+00,1.000


In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression

#选择K个最好的特征，返回选择特征后的数据



selector=SelectKBest(f_regression, k=2)
X2=selector.fit_transform(X_, Y)
X2_idx=selector.get_support(indices=True)
X2 = pd.DataFrame(X2,columns=[X2_idx])
X2

In [ ]:
X2_idx

In [ ]:
from sklearn.feature_selection import mutual_info_regression

#选择K个最好的特征，返回选择特征后的数据

Y=Y.ravel()
selector=SelectKBest(mutual_info_regression, k=5)
X3=selector.fit_transform(X_, Y)
X3_idx=selector.get_support(indices=True)
X3 = pd.DataFrame(X3,columns=[X3_idx])
X3_idx


In [ ]:
X3_idx